<a href="https://colab.research.google.com/github/Satwikram/NLP-Implementations/blob/main/BERT/End%20to%20End%20BERT%20using%20TFX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Author: Satwik Ram


**End to End BERT Implementation using TFX Pipelines**

### Setup

In [ ]:
!pip install tfx
!pip install transformers

### Connecting to Kaggle

In [1]:
from google.colab import files

files.upload()

! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [2]:
!kaggle competitions download -c sentiment-analysis-on-movie-reviews
!unzip /content/sentiment-analysis-on-movie-reviews.zip

  0% 0.00/1.90M [00:00<?, ?B/s]
100% 1.90M/1.90M [00:00<00:00, 43.2MB/s]
Archive:  /content/sentiment-analysis-on-movie-reviews.zip
  inflating: sampleSubmission.csv    
  inflating: test.tsv.zip            
  inflating: train.tsv.zip           


In [3]:
!unzip /content/test.tsv.zip
!unzip /content/train.tsv.zip

Archive:  /content/test.tsv.zip
  inflating: test.tsv                
Archive:  /content/train.tsv.zip
  inflating: train.tsv               


In [4]:
# !sudo rm -r /content/tfx

### Importing Dependencies

In [5]:
import os
import pprint
import numpy as np
import tempfile
import urllib

import absl
import pandas as pd
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

import tfx
from tfx.components import CsvExampleGen
from typing import Dict, List, Text
from tfx.components import Evaluator
from tfx.components import Pusher
# from tfx.components import ResolverNode

from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing
# from tfx.utils.dsl_utils import external_input


from tfx.proto import example_gen_pb2
from tfx.components import ImportExampleGen

from tfx.components import StatisticsGen
import tensorflow_data_validation as tfdv
from tfx.components import SchemaGen
from tfx.components import ExampleValidator
from tfx.components import Transform

from sklearn.model_selection import train_test_split

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

### Loading Dataset

In [6]:
df = pd.read_csv("/content/train.tsv", sep = "\t")

In [7]:
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [8]:
df = df.drop_duplicates(subset=['SentenceId'], keep='first')

In [9]:
df.to_csv("data.csv", index=False)

### Setup Pipeline Paths

In [10]:
!cd /content/
os.makedirs("/content/tfx/", exist_ok=True)
os.makedirs("/content/tfx/pipelines", exist_ok=True)
os.makedirs("/content/tfx/metadata", exist_ok=True)
os.makedirs("/content/tfx/logs", exist_ok=True)
os.makedirs("/content/tfx/data", exist_ok=True)
os.makedirs("/content/tfx/serving_model", exist_ok=True)

In [11]:
_tfx_root = os.path.join(os.getcwd(), 'tfx');        # Create location ~/tfx
_pipeline_root = os.path.join(_tfx_root, 'pipelines');      # Join ~/tfx/pipelines/
_metadata_db_root = os.path.join(_tfx_root, 'metadata.db');    # Join ~/tfx/metadata.db
_log_root = os.path.join(_tfx_root, 'logs');
_model_root = os.path.join(_tfx_root, 'model');
_data_root = os.path.join(_tfx_root, 'data');
_serving_model_dir = os.path.join(_tfx_root, 'serving_model')
_data_filepath = os.path.join(_data_root, "data_trans.csv")

_input_fn_module_file = 'inputfn_trainer.py'
_constants_module_file = 'constants_trainer.py'
_model_trainer_module_file = 'model_trainer.py'
_transform_fn_module_file = 'transform_fn.py'

### Data Ingestion

In [12]:
df = pd.read_csv('/content/data.csv')

##Drop useless columns
df.drop(['PhraseId', 'SentenceId',], axis=1, inplace=True)

#Drop NA rows
df.dropna(inplace=True)

##Keep a test set for final testing( TFX internally splits train and validation data )
traindf, testdf = train_test_split(df, test_size=0.1, random_state=42, shuffle=True)


print(len(traindf))
print(len(testdf))

traindf.to_csv("/content/tfx/data/data_trans.csv", index=False, header=True)
testdf.to_csv("test.csv", index=False, header=False)

7676
853


In [13]:
!head {_data_filepath}

Phrase,Sentiment
"With the dog days of August upon us , think of this dog of a movie as the cinematic equivalent of high humidity .",1
"One of the smarter offerings the horror genre has produced in recent memory , even if it 's far tamer than advertised .",3
A poignant comedy that offers food for thought .,4
Completely awful Iranian drama ... as much fun as a grouchy ayatollah in a cold mosque .,0
"The film is enriched by an imaginatively mixed cast of antic spirits , headed by Christopher Plummer as the subtlest and most complexly evil Uncle Ralph I 've ever seen in the many film and stage adaptations of the work .",4
Beautifully reclaiming the story of Carmen and recreating it an in an African idiom .,3
Kinnear ... gives his best screen performance with an oddly winning portrayal of one of life 's ultimate losers .,4
Perhaps the film should be seen as a conversation starter .,2
"It 's far from a frothy piece , and the characters are complex , laden with plenty of baggage and tinged w

### Create the InteractiveContext

In [14]:
context = InteractiveContext(pipeline_root=_tfx_root)

### ExampleGen

In [15]:
example_gen = CsvExampleGen(input_base=_data_root)
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

### Examining the output of ExampleGen

In [16]:
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval"] /content/tfx/CsvExampleGen/examples/1


### Looking at the first 3 examples

In [17]:
# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "Phrase"
    value {
      bytes_list {
        value: "With the dog days of August upon us , think of this dog of a movie as the cinematic equivalent of high humidity ."
      }
    }
  }
  feature {
    key: "Sentiment"
    value {
      int64_list {
        value: 1
      }
    }
  }
}

features {
  feature {
    key: "Phrase"
    value {
      bytes_list {
        value: "One of the smarter offerings the horror genre has produced in recent memory , even if it \'s far tamer than advertised ."
      }
    }
  }
  feature {
    key: "Sentiment"
    value {
      int64_list {
        value: 3
      }
    }
  }
}

features {
  feature {
    key: "Phrase"
    value {
      bytes_list {
        value: "A poignant comedy that offers food for thought ."
      }
    }
  }
  feature {
    key: "Sentiment"
    value {
      int64_list {
        value: 4
      }
    }
  }
}



### Data Analysis

In [18]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [19]:
context.show(statistics_gen.outputs['statistics'])

### Schmea

In [20]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [21]:
schema_gen

SchemaGen(spec: <tfx.types.standard_component_specs.SchemaGenSpec object at 0x7f3b928fea10>, executor_spec: <tfx.dsl.components.base.executor_spec.ExecutorClassSpec object at 0x7f3b928fe290>, driver_class: <class 'tfx.dsl.components.base.base_driver.BaseDriver'>, component_id: SchemaGen, inputs: {'statistics': OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={})}, outputs: {'schema': OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={})})

In [22]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Phrase',BYTES,required,single,-
'Sentiment',INT,required,single,-


### Example Validator

In [23]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [24]:
context.show(example_validator.outputs['anomalies'])

### Data Transform

In [25]:
%%writefile {_transform_fn_module_file}

from transformers import AutoTokenizer, BertTokenizer
from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf

def preprocessing_fn(inputs):
  """Preprocess input column of text into transformed columns of.
      * input token ids
      * input mask
      * input type ids
  """

  checkpoint = "bert-base-uncased"
  sequence_length = 512

  tokenizer = AutoTokenizer.from_pretrained(checkpoint)

  def tokenize_text(text, sequence_length=sequence_length):

    """
    Perform the BERT preprocessing from text -> input token ids and attention mask
    """
    
    tokens = tokenizer.encode_plus(
        text,
        max_length=sequence_length,
        truncation=True,
        padding="max_length",
        add_special_tokens=True,
        return_tensors="np"
    )

    return (
            tokens["input_ids"], 
            tokens["attention_mask"]
    )

    input_ids, attention_mask = tokenize_text(inputs["Phrase"])

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'label': inputs['Sentiment']
    }

Writing transform_fn.py


In [ ]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath("transform_fn.py")
    )
context.run(transform)

In [ ]:
##Test the input function
import inputfn_trainer as ift
import tensorflow as tf

#Test dataset read + Feat Engg function's - output's CSV + Feature engg columns
eval_file = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'eval/*')
fn_d = ift.make_input_fn(dir_uri = eval_file,
                    mode = tf.estimator.ModeKeys.EVAL,
                    # vnum_epochs = 1,
                    batch_size = 10)

fn_d(v_test=True)

In [ ]:
# transform = tfx.components.Transform(
#     examples=example_gen.outputs['examples'],
#     schema=schema_gen.outputs['schema'],
#     module_file=os.path.abspath(_transform_fn_module_file))
# context.run(transform, enable_cache=True)